## IMDb Animated Movie Data Collection/Cleaning

Author: **Michael B (MSB46)**

## Objective:

The purpose of this notebook is to fix the spreadsheet of values that may have been missing or inaccurate since the initial data scraping process. This includes reformatting certain columns so that they're in a more appropiate data type.

When data is retrieved through a scrapper like BeautifulSoup or Selenium, much of the data will be read by the scrapper as an object type at first. Features like metacritic score and votescore are a lot easier to be detected as an integer but a more complicated example would would be worldwide gross or budget estimate. This is because as long as there are characters that are non-digits in the text (think the currency that precede the number or the letters that may go after it), it will be automatically seen as a non-numeric string.

Changing data types into a numeric type is done so that it's easier to visualize our data using a plotting tool like matplotlib or seaborn which will be done in the EDA section.

As such, for many columns like gross, budget, and votes, our main goal is to covert these numbers into a numeric type by removing/reusing the non-numeric characters that are present in a cell value.

I also intend on making multiple datasets that focus more on a specific aspect of a movie like directors, production companies, and actors involved. This is to ease the process of viewing specific fields in Tableau more efficiently (at least in my perspective). Additional feautures might also be created but most fields will probably be made in Tableau through field calculations.

In [ ]:
!pip install CurrencyConverter

In [ ]:
!pip install PyCurrency_Converter

In [ ]:
import PyCurrency_Converter as pyc

In [ ]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)

df = pd.read_csv("/content/imdb_animated_movies_uncleaned.csv")
df.head()

,title,year,rating,runtime,votes,votescore,metacritic,budget,opening_na,worldwide,story,genres,origin,languages,companies,release_date,writers,director,cast,crew_count
0,5 Centimeters per Second,2007,TV-PG,1 hour 3 minutes,63K,7.5,NaN,"$5,000,000 (estimated)",NaN,"$493,937","Told in three interconnected segments, Takaki ...","Animation, Drama, Family, Romance",Japan,Japanese,CoMix Wave,"March 3, 2007 (Japan)",Makoto Shinkai,Makoto Shinkai,"Kenji Mizuhashi, Yoshimi Kondou, Satomi Hanamu...",132
1,9,2009,PG-13,1 hour 19 minutes,146K,7.0,60.0,"$30,000,000 (estimated)","$10,740,446","$48,428,063",A rag doll that awakens in a postapocalyptic f...,"Animation, Action, Adventure, Drama, Fantasy, ...","United States, Canada, Luxembourg",English,"Focus Features, Relativity Media, Arc Productions","September 9, 2009 (United States)","Pamela Pettler, Shane Acker, Ben Gluck",Shane Acker,"Christopher Plummer, Martin Landau, John C. Re...",382
2,A Boy Named Charlie Brown,1969,G,1 hour 26 minutes,5.6K,7.3,NaN,"$1,100,000 (estimated)",NaN,NaN,Charlie Brown makes his way to the national sp...,"Animation, Comedy, Drama, Family, Musical",United States,English,"Cinema Center Films, Lee Mendelson Film Produc...","December 4, 1969 (United States)",Charles M. Schulz,Bill Melendez,"Peter Robbins, Pamelyn Ferdin, Glenn Gilger, A...",76
3,A Bug's Life,1998,G,1 hour 35 minutes,311K,7.2,78.0,"$120,000,000 (estimated)","$291,121","$363,258,859","A misfit ant, looking for ""warriors"" to save h...","Animation, Adventure, Comedy, Family","United States, Japan","English, Russian","Pixar Animation Studios, Walt Disney Pictures","November 25, 1998 (United States)","John Lasseter, Andrew Stanton, Joe Ranft","John Lasseter, Andrew Stanton","David Foley, Kevin Spacey, Julia Louis-Dreyfus...",670
4,A Cat in Paris,2010,PG,1 hour 10 minutes,12K,6.9,63.0,NaN,"$34,554","$2,082,071","In Paris, a cat who lives a secret life as a c...","Animation, Adventure, Comedy, Crime, Family","France, Belgium","French, English","Folimage, Centre du Cinéma et de l'Audiovisuel...","December 15, 2010 (Belgium)","Alain Gagnol, Jacques-Rémy Girerd","Jean-Loup Felicioli, Alain Gagnol","Dominique Blanc, Bernadette Lafont, Bruno Salo...",125


In [ ]:
df.columns

Index(['title', 'year', 'rating', 'runtime', 'votes', 'votescore',
       'metacritic', 'budget', 'opening_na', 'worldwide', 'story', 'genres',
       'origin', 'languages', 'companies', 'release_date', 'writers',
       'director', 'cast', 'crew_count'],
      dtype='object')

In [ ]:
len(df.columns)

20

In [ ]:
df.isnull().any()

title           False
year            False
rating          False
runtime         False
votes           False
votescore       False
metacritic       True
budget           True
opening_na       True
worldwide        True
story           False
genres          False
origin          False
languages       False
companies        True
release_date    False
writers          True
director        False
cast             True
crew_count      False
dtype: bool

### Handling null values for each column
* Rating -> "Not Rated"
* Metacritic -> -1
* Any column involving currency -> -1
* Production Companies -> "Unnamed"
* Aspect Ratio -> N/A
* Writers -> "Unspecified"

In [ ]:
df.dtypes

title            object
year              int64
rating           object
runtime          object
votes            object
votescore       float64
metacritic      float64
budget           object
opening_na       object
worldwide        object
story            object
genres           object
origin           object
languages        object
companies        object
release_date     object
writers          object
director         object
cast             object
crew_count        int64
dtype: object

### Marking currencies
IMDb typically uses the country a movie originates from as a reference for the currency used when specifying the budget's estimate. The goal here is to detect as many currencies found as possible so they can be used as another feature for the final dataframe. It's important to note that currency conversions may not be completely accurate since the moment you are reading this.

In [ ]:
from currency_converter import CurrencyConverter
import PyCurrency_Converter as pyc
from datetime import date
c = CurrencyConverter(fallback_on_wrong_date=True, fallback_on_missing_rate=True, decimal=False)

In [ ]:
def trim_budget_string(x):
    if type(x) == float:
        return x
    else:
        return x.split(" (")[0].replace(" ","")


In [ ]:
df['budget'] = df['budget'].apply(trim_budget_string)

In [ ]:
def find_currency_symbol(x):
    if type(x) == float:
        return ''
    if x[0] == '£':
        return 'euro'
    elif x[0] == '¥':
        return 'yen'
    elif x[0] == '€':
        return 'pound'
    elif x[:2] == 'A$':
        return 'austrailian dollar'
    if x[:3] == "CN¥":
        return 'chinese yuan'
    elif x[:3] == "DKK":
        return 'danish krone'
    elif x[:3] == "PLN":
        return 'poland zloty'
    elif x[:3] == "FRF":
        return 'french franc'
    elif x[:3] == "DEM":
        return 'deutschmark'
    elif x[:3] == "NOK":
        return 'norwegian krone'
    elif x[:3] == "SEK":
        return 'swedish krona'
    else:
        return 'usd'


In [ ]:
df['currency'] = df['budget'].apply(find_currency_symbol)

In [ ]:
df['currency'].unique()

array(['usd', '', 'pound', 'yen', 'deutschmark', 'chinese yuan',
       'norwegian krone', 'austrailian dollar', 'swedish krona',
       'danish krone', 'euro', 'poland zloty', 'french franc'],
      dtype=object)

### Removing currency mark from budget_est. Will be relocated to new column

In [ ]:
len3currency = ["CN¥", 'DKK', 'PLN', 'FRF', "DEM", "NOK", "SEK"]
len2currency = ['A$']

In [ ]:
def remove_currency_symbol(x):
    if type(x) == float:
        return x
    elif x[:3] in len3currency:
        return x[3:]
    elif x[:2] == 'A$':
        return x[2:]
    elif x != '':
        return x[1:]
    else:
        return ''

In [ ]:
df['budget'] = df['budget'].apply(remove_currency_symbol).replace(',','', regex=True)

In [ ]:
df['budget']

0        5000000
1       30000000
2        1100000
3      120000000
4            NaN
         ...    
680       250000
681     80000000
682    370000000
683     20000000
684    150000000
Name: budget, Length: 685, dtype: object

### Budget conversion to USD

In [ ]:
df['year']

0      2007
1      2009
2      1969
3      1998
4      2010
       ... 
680    1968
681    2010
682    2016
683    2004
684    2016
Name: year, Length: 685, dtype: int64

In [ ]:
df['currency'].unique()

array(['usd', '', 'pound', 'yen', 'deutschmark', 'chinese yuan',
       'norwegian krone', 'austrailian dollar', 'swedish krona',
       'danish krone', 'euro', 'poland zloty', 'french franc'],
      dtype=object)

In [ ]:
# c.currencies

In [ ]:
for x in range(len(df)):
        if df['currency'][x] == "" or df['currency'][x] == "nan":
            df['budget'][x] = ""

        elif df['currency'][x] == 'dollar':
          df['budget'][x] = int(df['budget'][x])

        elif df['currency'][x] == 'pound':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'GBP', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'euro':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'EUR', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'austrailian dollar':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'AUD', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'deutschmark':
          df['budget'][x] = round(int(df['budget'][x]) * 0.0025)

        elif df['currency'][x] == 'poland zloty':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'PLN', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'french franc':
          df['budget'][x] = round(int(df['budget'][x]) * 1.09)

        elif df['currency'][x] == 'chinese yuan':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'CNY', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'danish krone':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'DKK', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'swedish krona':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'SEK', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'norwegian krone':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'NOK', 'USD', date=date(df['year'][x], 1, 1))

        elif df['currency'][x] == 'yen':
          df['budget'][x] = c.convert(int(df['budget'][x]), 'JPY', 'USD', date=date(df['year'][x], 1, 1))

    # except Exception as e:
    #     print(f"Index: {x}\n{e}\n{df.iloc[x]}")

In [ ]:
df[['budget','currency']]

,budget,currency
0,5000000,usd
1,30000000,usd
2,1100000,usd
3,120000000,usd
4,,
...,...,...
680,294725.0,euro
681,80000000,usd
682,3081670.777644,yen
683,20000000,usd


In [ ]:
df.head(3)

,title,year,rating,runtime,votes,votescore,metacritic,budget,opening_na,worldwide,story,genres,origin,languages,companies,release_date,writers,director,cast,crew_count,currency
0,5 Centimeters per Second,2007,TV-PG,1 hour 3 minutes,63K,7.5,NaN,5000000,NaN,"$493,937","Told in three interconnected segments, Takaki ...","Animation, Drama, Family, Romance",Japan,Japanese,CoMix Wave,"March 3, 2007 (Japan)",Makoto Shinkai,Makoto Shinkai,"Kenji Mizuhashi, Yoshimi Kondou, Satomi Hanamu...",132,usd
1,9,2009,PG-13,1 hour 19 minutes,146K,7.0,60.0,30000000,"$10,740,446","$48,428,063",A rag doll that awakens in a postapocalyptic f...,"Animation, Action, Adventure, Drama, Fantasy, ...","United States, Canada, Luxembourg",English,"Focus Features, Relativity Media, Arc Productions","September 9, 2009 (United States)","Pamela Pettler, Shane Acker, Ben Gluck",Shane Acker,"Christopher Plummer, Martin Landau, John C. Re...",382,usd
2,A Boy Named Charlie Brown,1969,G,1 hour 26 minutes,5.6K,7.3,NaN,1100000,NaN,NaN,Charlie Brown makes his way to the national sp...,"Animation, Comedy, Drama, Family, Musical",United States,English,"Cinema Center Films, Lee Mendelson Film Produc...","December 4, 1969 (United States)",Charles M. Schulz,Bill Melendez,"Peter Robbins, Pamelyn Ferdin, Glenn Gilger, A...",76,usd


### Converting all monetary values to integer

In [ ]:
def convert_int(s):
    return s.apply(lambda x: re.sub(',','',x[1:-1]) if not(x == "") else "").replace('','-1').astype(int)

In [ ]:
df['worldwide'] = df['worldwide'].apply(remove_currency_symbol).replace(',','', regex=True)

In [ ]:
df['opening_na'] = df['opening_na'].apply(remove_currency_symbol).replace(',','', regex=True)

### Getting profit data

In [ ]:
df.opening_na

0           NaN
1      10740446
2           NaN
3        291121
4         34554
         ...   
680      107105
681    16411322
682     1813781
683     9485494
684    75063401
Name: opening_na, Length: 685, dtype: object

In [ ]:
df.insert(len(df.columns)-1, 'profit', "")
for x in range(len(df)):
  try:
    if (df['budget'][x] in ['', float('nan')]) or (df['worldwide'][x] in ['', float('nan')]):
        df['profit'][x] = ""
    else:
        df['profit'][x] = str(round(float(df['worldwide'][x]) - int(df['budget'][x])))
  except ValueError:
    continue

In [ ]:
df.profit

0       -4506063
1       18428063
2               
3      243258859
4               
         ...    
680       978536
681    123509374
682    379156511
683      9266490
684    875521689
Name: profit, Length: 685, dtype: object

### Converting more columns to appropriate data types

In [ ]:
df['votes'] = df['votes'].replace(',','', regex=True)
df['votes'] = df['votes'].apply(lambda x: int(float(x[:-1]) * 1_000_000) if x[-1] == 'M' else int(float(x[:-1]) * 1_000))

In [ ]:
df['votes']

0       63000
1      146000
2        5600
3      311000
4       12000
        ...  
680     28000
681     23000
682    311000
683      9000
684    538000
Name: votes, Length: 685, dtype: int64

In [ ]:
df['year'].astype(str)

0      2007
1      2009
2      1969
3      1998
4      2010
       ... 
680    1968
681    2010
682    2016
683    2004
684    2016
Name: year, Length: 685, dtype: object

In [ ]:
df.year.unique()

array([2007, 2009, 1969, 1998, 2010, 2000, 2008, 1995, 2011, 2006, 2019,
       2016, 1994, 2020, 2012, 1988, 1992, 1951, 2015, 1989, 1996, 2021,
       1981, 1986, 1991, 1997, 1954, 2022, 2004, 2014, 1985, 1968, 1967,
       2018, 2001, 1942, 1983, 2017, 1993, 1971, 1973, 2013, 2023, 2003,
       2005, 1950, 1990, 1941, 2002, 1940, 1999, 1972, 1947, 1987, 1974,
       1955, 1979, 1948, 1984, 1961, 1977, 1953, 1959, 1937, 1946, 1949,
       1926, 1970, 1964, 1980, 1982, 1978, 1975, 1963, 1944, 1976])

In [ ]:
df['year'] = df['year'].astype(int)

In [ ]:
df.insert(2, "month", "")
df.insert(2, "day", "")

df['month'] = df['release_date'].apply(lambda x: x.split()[0] if len(x.split()) > 3 else '')
df['day'] = df['release_date'].apply(lambda x: x.split()[1][0]
                                     if len(x.split()) > 3 else '')

In [ ]:
df['release_date'] = df['release_date'].apply(lambda x: " ".join(x.split()[:3] if len(x.split()) > 3 else ''))

### Production companies

In [ ]:
df['companies'].fillna('Unnamed',inplace=True)
df['companies'] = df['companies'].apply(lambda x: x.split(', '))

df.insert(list(df.columns).index('companies'), "company_count", 0)
df['company_count'] =  df['companies'].apply(lambda x: len(x))

In [ ]:
df['companies'] = df['companies'].apply(lambda x: ", ".join(x))
df['companies'].head()

0                                           CoMix Wave
1    Focus Features, Relativity Media, Arc Productions
2    Cinema Center Films, Lee Mendelson Film Produc...
3        Pixar Animation Studios, Walt Disney Pictures
4    Folimage, Centre du Cinéma et de l'Audiovisuel...
Name: companies, dtype: object

In [ ]:
df[['companies','company_count']].head(5)

,companies,company_count
0,CoMix Wave,1
1,"Focus Features, Relativity Media, Arc Productions",3
2,"Cinema Center Films, Lee Mendelson Film Produc...",3
3,"Pixar Animation Studios, Walt Disney Pictures",2
4,"Folimage, Centre du Cinéma et de l'Audiovisuel...",3


### Countries and Languages

In [ ]:
df.insert(list(df.columns).index('origin'),'country_count', 0)
df.insert(list(df.columns).index('languages'),'language_count', 0)
df['languages'].fillna("", inplace=True)

for x in range(len(df)):
    try:
        df['country_count'][x] = df['origin'][x].count(",")+1
        df['language_count'][x] = df['languages'][x].count(",")+1
    except:
        print(df['origin'][x], df['languages'][x])

df['origin'] = df['origin'].apply(lambda x: x.split(', '))
df['languages'] = df['languages'].apply(lambda x: x.split(', '))

In [ ]:
df[['origin','country_count','languages','language_count','title']]

,origin,country_count,languages,language_count,title
0,[Japan],1,[Japanese],1,5 Centimeters per Second
1,"[United States, Canada, Luxembourg]",3,[English],1,9
2,[United States],1,[English],1,A Boy Named Charlie Brown
3,"[United States, Japan]",2,"[English, Russian]",2,A Bug's Life
4,"[France, Belgium]",2,"[French, English]",2,A Cat in Paris
...,...,...,...,...,...
680,"[United Kingdom, United States]",2,[English],1,Yellow Submarine
681,"[United States, New Zealand]",2,[English],1,Yogi Bear
682,[Japan],1,[Japanese],1,Your Name.
683,"[Japan, United States]",2,"[Japanese, English]",2,Yu-Gi-Oh!: The Movie - Pyramid of Light


### Genres

In [ ]:
df['genres'] = df['genres'].replace("Animation, ","", regex=True).apply(lambda x: x.split(', '))

In [ ]:
df['genres'].head()

0                             [Drama, Family, Romance]
1    [Action, Adventure, Drama, Fantasy, Sci-Fi, Th...
2                     [Comedy, Drama, Family, Musical]
3                          [Adventure, Comedy, Family]
4                   [Adventure, Comedy, Crime, Family]
Name: genres, dtype: object

In [ ]:
df.insert(list(df.columns).index('genres'), "genre_count", 0)

for x in range(len(df)):
    # Genres not including Animation
    df['genre_count'][x] = len(df['genres'][x])

df['genre_count'].head()

0    3
1    6
2    4
3    3
4    4
Name: genre_count, dtype: int64

In [ ]:
df['story'].fillna("",inplace=True)

In [ ]:
df['genres'] = df['genres'].apply(lambda x: ", ".join(x))
df['genres'].head()

0                               Drama, Family, Romance
1    Action, Adventure, Drama, Fantasy, Sci-Fi, Thr...
2                       Comedy, Drama, Family, Musical
3                            Adventure, Comedy, Family
4                     Adventure, Comedy, Crime, Family
Name: genres, dtype: object

In [ ]:
df.insert(list(df.columns).index('story'), "story_word_count", 0)

for x in range(len(df)):
    if df['story'][x] == "":
        continue
    df['story_word_count'][x] = df['story'][x].count(" ")+1

### Runtime

In [ ]:
df.runtime

0       1 hour 3 minutes
1      1 hour 19 minutes
2      1 hour 26 minutes
3      1 hour 35 minutes
4      1 hour 10 minutes
             ...        
680    1 hour 25 minutes
681    1 hour 20 minutes
682    1 hour 46 minutes
683    1 hour 30 minutes
684    1 hour 48 minutes
Name: runtime, Length: 685, dtype: object

In [ ]:
def runtime_to_minutes(s):
        t = s.split()
        hour = t[0]
        if t[1] == "minutes":
            return int(t[0])
        try:
            minutes = t[2]
        except IndexError:
            minutes = 0
        total_minutes = (int(hour) * 60) + int(minutes)
        return total_minutes

In [ ]:
df['runtime'] = df['runtime'].apply(runtime_to_minutes)
df['runtime'].head()

0    63
1    79
2    86
3    95
4    70
Name: runtime, dtype: int64

In [ ]:
df.columns

Index(['title', 'year', 'day', 'month', 'rating', 'runtime', 'votes',
       'votescore', 'metacritic', 'budget', 'opening_na', 'worldwide',
       'story_word_count', 'story', 'genre_count', 'genres', 'country_count',
       'origin', 'language_count', 'languages', 'company_count', 'companies',
       'release_date', 'writers', 'director', 'cast', 'crew_count', 'profit',
       'currency'],
      dtype='object')

In [ ]:
df['currency'].fillna("", inplace=True)
df['profit'].fillna("", inplace=True)
df['budget'].fillna("", inplace=True)
df['worldwide'].fillna("", inplace=True)
df['opening_na'].fillna("", inplace=True)

### Directors/Writers

It was interesting to see that there is only one movie in the dataframe where there isn't any credited writer. So to solve this, I chose to convert any null value inside the column into the placeholder "Unspecified" value.

In [ ]:
# insert column with insert(location, column_name, column_value)
wrt_column = df.pop("writers")
df.insert(len(df.columns)-1, "writers", wrt_column)

dir_column = df.pop("director")
df.insert(len(df.columns)-1, "directors", dir_column)

In [ ]:
df['directors'] = df['directors'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", str(x)) if str(x).strip() != "nan" else "Unspecified")
df['writers'] = df['writers'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", str(x)) if str(x).strip() != "nan" else "Unspecified")

In [ ]:
# df[(df['directors'].str.contains("Zack")) | (df['writers'].str.contains("Zack"))]

In [ ]:
df['directors']

0                            Makoto Shinkai
1                               Shane Acker
2                             Bill Melendez
3             John Lasseter, Andrew Stanton
4         Jean-Loup Felicioli, Alain Gagnol
                       ...                 
680                          George Dunning
681                             Eric Brevig
682                          Makoto Shinkai
683                           Hatsuki Tsuji
684    Byron Howard, Rich Moore, Jared Bush
Name: directors, Length: 685, dtype: object

In [ ]:
# list(df[df['genres'].str.contains("Drama")].sort_values(by='votes', ascending=False)['title'].values)[:10]

In [ ]:
# list(df[df['directors'].str.contains("Brad Bird")].sort_values(by='year')['title'].values)

In [ ]:
print(df[df['directors'].str.contains("Brad Bird")]['year'].min())
print(df[df['directors'].str.contains("Brad Bird")]['year'].max())

1999
2018


In [ ]:
import re
# https://stackoverflow.com/a/14599280

def getFreq(col, str_name, df_=df, include_movie_list=False, get_full_list=False, include_years_active=False):
    countDict = {}
    moviesDict = {}
    earliest, latest = {}, {}

    # Dictionary counts frequencies
    for c in df_[col]:
      if isinstance(c,float):
        print("Whoops! Float value found", c)
        continue
      try:
        for x in c.split(', '):
          # print(x)
          if include_movie_list:
            if get_full_list:
              moviesDict[x] = list(df_.fillna("")[df_.fillna("")[col].str.contains(re.sub("[\(\[].*?[\)\]]", "", x))].sort_values(by='votes', ascending=False)['title'].values)
            else:
              moviesDict[x] = list(df_.fillna("")[df_.fillna("")[col].str.contains(re.sub("[\(\[].*?[\)\]]", "", x))].sort_values(by='votes', ascending=False)['title'].values)[:15]
          if include_years_active:
            earliest[x] = df_.fillna("")[df_.fillna("")[col].str.contains(x)]['year'].min()
            latest[x] = df_.fillna("")[df_.fillna("")[col].str.contains(x)]['year'].max()
          val = countDict.get(x.strip(), 0)
          countDict[x] = val + 1

      except AttributeError:
        for x in c:
          # print(x)
          if include_movie_list:
            if get_full_list:
              moviesDict[x] = list(df_.fillna("")[df_.fillna("")[col].str.contains(re.sub("[\(\[].*?[\)\]]", "", x))].sort_values(by='votes', ascending=False)['title'].values)
            else:
              moviesDict[x] = list(df_.fillna("")[df_.fillna("")[col].str.contains(re.sub("[\(\[].*?[\)\]]", "", x))].sort_values(by='votes', ascending=False)['title'].values)[:15]
          if include_years_active:
            earliest[x] = df_[df_[col].str.contains(x)]['year'].min()
            latest[x] = df_[df_[col].str.contains(x)]['year'].max()
          val = countDict.get(x.strip(), 0)
          countDict[x] = val + 1

    new_dataframe = pd.DataFrame.from_dict(data=countDict, columns=[str_name], orient="index")
    if include_movie_list:
      new_dataframe[f'movies_{col}'] = new_dataframe.index.map(lambda x: moviesDict[x])
    if include_years_active:
      new_dataframe[f'years_active'] = new_dataframe.index.map(lambda x: f"{earliest[x]}-{latest[x]}")

    return new_dataframe, countDict

In [ ]:
df_dir, dict_dir = getFreq(col='directors', str_name="times_directed", include_movie_list=True)
df_wrt, dict_wrt = getFreq(col='writers', str_name="times_written", include_movie_list=True)
df_dir_wrt = df_dir.join(df_wrt, how="outer")

In [ ]:
# df_wrt.loc["A.A. Milne"]

In [ ]:
df_dir_wrt.fillna(0, inplace=True)
df_dir_wrt['times_directed'] = df_dir_wrt['times_directed'].astype(int)
df_dir_wrt['times_written'] = df_dir_wrt['times_written'].astype(int)

In [ ]:
df_dir_wrt

,times_directed,movies_directors,times_written,movies_writers
A.A. Milne,0,0,4,"[The Many Adventures of Winnie the Pooh, The T..."
Aaron Blabey,0,0,1,[The Bad Guys]
Aaron Blaise,1,[Brother Bear],0,0
Aaron Horvath,2,"[The Super Mario Bros. Movie, Teen Titans GO! ...",1,[Teen Titans GO! To the Movies]
Adam Balsam,0,0,1,[Legends of Oz: Dorothy's Return]
...,...,...,...,...
Yûsuke Watanabe,0,0,1,[Dragon Ball Z: Battle of Gods]
Zach Lewis,0,0,1,[Klaus]
Zack Snyder,1,[Legend of the Guardians: The Owls of Ga'Hoole],0,0
Zeb Wells,0,0,1,[Hell and Back]


In [ ]:
df['directors'] = df['directors'].str.rstrip(", ").apply(lambda x: x.split(', '))
df['writers'] = df['writers'].str.rstrip(", ").apply(lambda x: x.split(', '))

In [ ]:
df.insert(list(df.columns).index('directors'), "writer_count", 0)
df.insert(list(df.columns).index('writers'), "director_count", 0)

for x in range(len(df)):
    df['director_count'][x] = len(df['directors'][x])
    df['writer_count'][x] = len(df['writers'][x])

# df['directors'] = df['directors'].apply(lambda x: ", ".join(x))
# df['writers'] = df['writers'].apply(lambda x: ", ".join(x))

### Result

In [ ]:
df.dtypes

title                object
year                  int64
day                  object
month                object
rating               object
runtime               int64
votes                 int64
votescore           float64
metacritic          float64
budget               object
opening_na           object
worldwide            object
story_word_count      int64
story                object
genre_count           int64
genres               object
country_count         int64
origin               object
language_count        int64
languages            object
company_count         int64
companies            object
release_date         object
cast                 object
crew_count            int64
profit               object
director_count        int64
writers              object
writer_count          int64
directors            object
currency             object
dtype: object

In [ ]:
df.to_csv("imdb_animated_movies_clean.csv", index = False)

After concluding the data collection / cleaning portion of the project, every feature should have a more appropiate data type, which means I can visualize my data more efficiently in the next part of the project where I begin to perform exploratory data analysis on the recently cleaned data.

### Dataset - Director, Writer

In [ ]:
def avg_score_by_keys(df, column, keys):
    score_by_col = {}
    ww_by_col = {}
    counter_a, counter_b  = {}, {}

    for x in range(len(df[[column,'votescore','worldwide']])):
        for k in keys:
          if isinstance(df[column][x], float):
              continue
          if k in df[column][x]:
              # VOTESCORE
              counter_a[k] = counter_a.get(k,0)
              val = score_by_col.get(k,0)
              score_by_col[k] = val + df['votescore'][x]
              counter_a[k] += 1

              # WORLDWIDE GROSS
              if df['worldwide'][x] == '':
                  continue
              else:
                  counter_b[k] = counter_b.get(k,0)
                  val_ww = ww_by_col.get(k,0)
                  ww_by_col[k] = val_ww + float(df['worldwide'][x])
                  counter_b[k] = counter_b.get(k) + 1

#     print(f'val:{val}, counter_a:{counter_a}')
#     print(mc_by_col)
    sum_ww = ww_by_col.copy()

    for k in keys:
        score_by_col[k] = round((score_by_col.get(k,0) / counter_a.get(k,1)),3)


        if counter_b.get(k) is not None and ww_by_col.get(k) is not None:
            ww_by_col[k] = round(ww_by_col.get(k,0) / counter_b.get(k,1), 3)

    return score_by_col, ww_by_col, sum_ww

In [ ]:
# def avg_score_by_keys(df_old, column, keys):
#     score_by_col = {}
#     ww_by_col = {}
#     sum_ww = {}

#     df_ = df_old.copy()
#     df_[df_['worldwide'] == ""] = 0
#     df_['worldwide'] = df_['worldwide'].astype(int)
#     df_[column] = df_[column].astype(str)

#     for k in keys:
#         score_by_col[k] = df_[df_[column].str.contains(k)]['votescore'].mean()
#         try:
#           ww_by_col[k] = df_[(df_[column].str.contains(k)) & (df_['worldwide'] != 0)]['worldwide'].mean()
#           sum_ww[k] = df_[(df_[column].str.contains(k)) & (df_['worldwide'] != 0)]['worldwide'].sum()
#         except KeyError as ke:
#           print(k, ke)
#           continue

#     return score_by_col, ww_by_col, sum_ww

In [ ]:
# df_['directors'] = df['directors'].astype(str)

In [ ]:
# df_[df_['directors'].str.contains('Aaron Blaise')]

In [ ]:
key_names = list(dict_dir.keys())
key_names.sort()
dir_avg_dict_vs, dir_avg_dict_ww, dir_sum_dict_ww = avg_score_by_keys(df, "directors", key_names)

key_names = list(dict_wrt.keys())
key_names.sort()
wrt_avg_dict_vs, wrt_avg_dict_ww, wrt_sum_dict_ww = avg_score_by_keys(df, "writers", key_names)

In [ ]:
# df[df['writers'].str.contains("Seth MacFarlane")]['votescore'].mean()

In [ ]:
wrt_avg_dict_vs['Seth MacFarlane']

7.5

In [ ]:
dir_avg_vs_df = pd.DataFrame.from_dict(data=dir_avg_dict_vs, columns=["avg_votescore_dir"], orient="index")
dir_avg_ww_df = pd.DataFrame.from_dict(data=dir_avg_dict_ww, columns=["avg_wwgross_dir"], orient="index")
dir_sum_ww_df = pd.DataFrame.from_dict(data=dir_sum_dict_ww, columns=["sum_wwgross_dir"], orient="index")

dir_df = dir_avg_vs_df.join(dir_avg_ww_df, how="outer").join(dir_sum_ww_df, how="outer")

wrt_avg_vs_df = pd.DataFrame.from_dict(data=wrt_avg_dict_vs, columns=["avg_votescore_wrt"], orient="index")
wrt_avg_ww_df = pd.DataFrame.from_dict(data=wrt_avg_dict_ww, columns=["avg_wwgross_wrt"], orient="index")
wrt_sum_ww_df = pd.DataFrame.from_dict(data=wrt_sum_dict_ww, columns=["sum_wwgross_wrt"], orient="index")

wrt_df = wrt_avg_vs_df.join(wrt_avg_ww_df, how="outer").join(wrt_sum_ww_df, how="outer")

df_dir_wrt = df_dir_wrt.join(dir_df, how="outer").join(wrt_df, how="outer")
df_dir_wrt

,times_directed,movies_directors,times_written,movies_writers,avg_votescore_dir,avg_wwgross_dir,sum_wwgross_dir,avg_votescore_wrt,avg_wwgross_wrt,sum_wwgross_wrt
A.A. Milne,0,0,4,"[The Many Adventures of Winnie the Pooh, The T...",NaN,NaN,NaN,6.575,70629593.0,211888779.0
Aaron Blabey,0,0,1,[The Bad Guys],NaN,NaN,NaN,6.800,250387888.0,250387888.0
Aaron Blaise,1,[Brother Bear],0,0,6.80,250397798.0,2.503978e+08,NaN,NaN,NaN
Aaron Horvath,2,"[The Super Mario Bros. Movie, Teen Titans GO! ...",1,[Teen Titans GO! To the Movies],6.85,707170045.0,1.414340e+09,6.700,52390236.0,52390236.0
Adam Balsam,0,0,1,[Legends of Oz: Dorothy's Return],NaN,NaN,NaN,5.000,21755418.0,21755418.0
...,...,...,...,...,...,...,...,...,...,...
Yûsuke Watanabe,0,0,1,[Dragon Ball Z: Battle of Gods],NaN,NaN,NaN,7.100,50461371.0,50461371.0
Zach Lewis,0,0,1,[Klaus],NaN,NaN,NaN,8.200,NaN,NaN
Zack Snyder,1,[Legend of the Guardians: The Owls of Ga'Hoole],0,0,6.90,140073390.0,1.400734e+08,NaN,NaN,NaN
Zeb Wells,0,0,1,[Hell and Back],NaN,NaN,NaN,5.400,157768.0,157768.0


In [ ]:
df_dir_wrt.insert(0, 'name', "")
df_dir_wrt['name'] = df_dir_wrt.index
df_dir_wrt.to_csv("animated_movies_writers_directors.csv", index = False)

### Dataset - Genre

In [ ]:
def create_df(column, index_name, dataframe=df, include_movies=(False, False), include_years=False):
  df_freq, dict_col = getFreq(col=column, str_name="freq", include_years_active=include_years, include_movie_list=include_movies[0], get_full_list=include_movies[1])
  keys_in_col = list(dict_col.keys())
  keys_in_col.sort()

  avg_dict_vs, avg_dict_ww, sum_dict_ww, = avg_score_by_keys(dataframe, column, keys_in_col)

  avg_vs_df = pd.DataFrame.from_dict(data=avg_dict_vs, columns=["avg_votescore"], orient="index")
  avg_ww_df = pd.DataFrame.from_dict(data=avg_dict_ww, columns=["avg_wwgross"], orient="index")
  sum_ww_df = pd.DataFrame.from_dict(data=sum_dict_ww, columns=["sum_wwgross"], orient="index")

  df_freq = df_freq.join(avg_vs_df, how="outer")

  df_final = df_freq.join(avg_ww_df, how="outer").join(sum_ww_df, how="outer")
  df_final.insert(0, index_name, "")
  df_final[index_name] = df_final.index
  df_final.to_csv(f"animated_movies_{column}.csv", index = False)

In [ ]:
create_df("genres","genre", include_movies=(True,False))

## Dataset - Companies

In [ ]:
create_df("companies","company", include_movies=(True,False), include_years=True)

## Dataset - Languages and Countries

In [ ]:
create_df("languages","language")
create_df("origin","country")

## Dataset - Actors

In [ ]:
create_df("cast","actor", include_movies=(True,False), include_years=True)

Whoops! Float value found nan
Whoops! Float value found nan
Whoops! Float value found nan
Whoops! Float value found nan
